In [1]:
import pandas as pd
import numpy as np
TOTAL_ZONAS = 10
TOTAL_MESES = 24
T = range(1,TOTAL_MESES)
Z = range(0,TOTAL_ZONAS) #de la zona 1 a la zona 5 son zona sur, de la 6 hasta la 10 zona norte

zona_norte_dim = np.array([2000, 3000]) #ancho x largo en metros
zona_sur_dim = np.array([1000, 3000]) #ancho x largo en metros
coordinadas_caneria_norte = np.array([1000,0])
coordinadas_caneria_sur= np.array([0,1500])


In [35]:
zonas_norte=[]
zonas_sur=[]
def generarZonasAleatoreas():
    for z in Z:
        if z < TOTAL_ZONAS / 2:
            zona_x = np.random.randint(zona_sur_dim[0])
            zona_y = np.random.randint(zona_sur_dim[1])
            zonas_sur.append({"coord": np.array([zona_x, zona_y])})
        else:
            zona_x = np.random.randint(zona_norte_dim[0])
            zona_y = np.random.randint(zona_norte_dim[1])
            zonas_norte.append({"coord": np.array([zona_x, zona_y])})


In [37]:
# maximos litros por metro cuadrado por mes que se pueden extraer en el centro de la zona
maximo_mes_zona_norte = {
    'enero': 0,
    'febrero': 0,
    'marzo': 6,
    'abril': 0,
    'mayo' : 3.8*30,
    'junio': 2.1*30,
    'julio': 5*30,
    'agosto': 10*30,
    'septiembre': 9*30,
    'octubre': 9.5*30,
    'noviembre': 5.8*30,
    'diciembre':0
}
maximo_mes_zona_sur = {
    'enero': 0,
    'febrero': 0,
    'marzo': 0,
    'abril': 0,
    'mayo' : 3.5*30,
    'junio': 1.5*30,
    'julio': 4.5*30,
    'agosto': 11.5*30,
    'septiembre': 10.7*30,
    'octubre': 11*30,
    'noviembre': 5*30,
    'diciembre':0
}
def calcularLitrosZona(x, ancho_total, maximo_mes, mes):
    centro_x = ancho_total / 2
    distancia = abs(x - centro_x)
    # Normalizamos la distancia (0 en el centro, 1 en el borde)
    factor = 1 - 0.3 * (distancia / (ancho_total / 2))  # el 0.3 controla cuánto baja en los bordes
    factor = max(factor, 0)  # evitar valores negativos
    return factor * maximo_mes[mes]

# Funciones por zona
def litrosPorMesNorte(x, y, mes):
    return calcularLitrosZona(x, zona_norte_dim[0], maximo_mes_zona_norte, mes)

def litrosPorMesSur(x, y, mes):
    return calcularLitrosZona(x, zona_sur_dim[0], maximo_mes_zona_sur, mes)

# Asignar producción mensual a cada zona
def asignarProduccionMensual():
    for zona in zonas_sur:
        zona["produccion"] = {}
        x, y = zona["coord"]
        for mes in maximo_mes_zona_sur.keys():
            zona["produccion"][mes] = litrosPorMesSur(x, y, mes)

    for zona in zonas_norte:
        zona["produccion"] = {}
        x, y = zona["coord"]
        for mes in maximo_mes_zona_norte.keys():
            zona["produccion"][mes] = litrosPorMesNorte(x, y, mes)

In [ ]:

generarZonasAleatoreas()
asignarProduccionMensual()

for zona in zonas_sur:
    coord = zona['coord']
    distancia = np.linalg.norm(coord - coordinadas_caneria_sur)
    zona['distancia_a_caneria'] = distancia

for zona in zonas_norte:
    coord = zona['coord']
    distancia = np.linalg.norm(coord - coordinadas_caneria_norte)
    zona['distancia_a_caneria'] = distancia




{'coord': array([ 577, 2199]), 'produccion': {'enero': np.float64(0.0), 'febrero': np.float64(0.0), 'marzo': np.float64(0.0), 'abril': np.float64(0.0), 'mayo': np.float64(100.149), 'junio': np.float64(42.921), 'julio': np.float64(128.763), 'agosto': np.float64(329.061), 'septiembre': np.float64(306.1698), 'octubre': np.float64(314.754), 'noviembre': np.float64(143.07), 'diciembre': np.float64(0.0)}, 'distancia_a_caneria': np.float64(906.3829212865829)}


In [ ]:
# GENERAR DATARAME, SE ASUME QUE LA PRIMERA MITAD DE COLUMNAS CORRESPONDE A LA ZONA SUR, EL RESTO A LA NORTE
# los meses son 1= enero y 12 = diciembre y se repite ciclicamente
TOTAL_MESES = 24 
columnas = ['z'] + [str(i) for i in range(1, TOTAL_MESES + 1)]
datos = []

nombres_meses = [
    'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
    'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'
]

for i, zona in enumerate(zonas_sur):
    produccion_anual = [zona["produccion"][mes] for mes in nombres_meses]

    produccion_repetida = (produccion_anual * (TOTAL_MESES // 12 + 1))[:TOTAL_MESES]
    fila = [i + 1] + produccion_repetida
    datos.append(fila)


offset = int(TOTAL_ZONAS / 2)
for i, zona in enumerate(zonas_norte):
    produccion_anual = [zona["produccion"][mes] for mes in nombres_meses]
    produccion_repetida = (produccion_anual * (TOTAL_MESES // 12 + 1))[:TOTAL_MESES]
    fila = [offset + i + 1] + produccion_repetida
    datos.append(fila)


df = pd.DataFrame(datos, columns=columnas)
df.index = range(1, TOTAL_ZONAS + 1)


df.head(10)


,z,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
1,1,0.0,0.0,0.0000,0.0,100.1490,42.9210,128.763,329.061,306.1698,...,0.0000,0.0,100.1490,42.9210,128.763,329.061,306.1698,314.7540,143.0700,0.0
2,2,0.0,0.0,0.0000,0.0,103.9290,44.5410,133.623,341.481,317.7258,...,0.0000,0.0,103.9290,44.5410,133.623,341.481,317.7258,326.6340,148.4700,0.0
3,3,0.0,0.0,0.0000,0.0,104.3700,44.7300,134.190,342.930,319.0740,...,0.0000,0.0,104.3700,44.7300,134.190,342.930,319.0740,328.0200,149.1000,0.0
4,4,0.0,0.0,0.0000,0.0,81.0600,34.7400,104.220,266.340,247.8120,...,0.0000,0.0,81.0600,34.7400,104.220,266.340,247.8120,254.7600,115.8000,0.0
5,5,0.0,0.0,0.0000,0.0,76.7760,32.9040,98.712,252.264,234.7152,...,0.0000,0.0,76.7760,32.9040,98.712,252.264,234.7152,241.2960,109.6800,0.0
6,6,0.0,0.0,5.4546,0.0,103.6374,57.2733,136.365,272.730,245.4570,...,5.4546,0.0,103.6374,57.2733,136.365,272.730,245.4570,259.0935,158.1834,0.0
7,7,0.0,0.0,4.5474,0.0,86.4006,47.7477,113.685,227.370,204.6330,...,4.5474,0.0,86.4006,47.7477,113.685,227.370,204.6330,216.0015,131.8746,0.0
8,8,0.0,0.0,4.9398,0.0,93.8562,51.8679,123.495,246.990,222.2910,...,4.9398,0.0,93.8562,51.8679,123.495,246.990,222.2910,234.6405,143.2542,0.0
9,9,0.0,0.0,5.7570,0.0,109.3830,60.4485,143.925,287.850,259.0650,...,5.7570,0.0,109.3830,60.4485,143.925,287.850,259.0650,273.4575,166.9530,0.0
10,10,0.0,0.0,5.3322,0.0,101.3118,55.9881,133.305,266.610,239.9490,...,5.3322,0.0,101.3118,55.9881,133.305,266.610,239.9490,253.2795,154.6338,0.0


In [2]:
import csv
with open('./output/c_t.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Precio por litro'])  # Encabezado opcional
    writer.writerow([1.31947])

In [8]:
df = pd.read_csv("./output/c_t.csv")
df_extraccion = pd.read_csv("./output/q_zt.csv")

In [ ]:
df[df['t'] == 5]['c_t'] #leer costos
df_extraccion[df_extraccion['z'] == 1]['6'].values[0] #leer extracion


np.float64(33.012)